<a href="https://colab.research.google.com/github/villa-lobos17/HeuristicSearch/blob/master/Razonamiento_y_planificacion_automatica_Busqueda_A_Estrella.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Resolver laberinto

Ejemplo de busqueda en estrella que se encarga de buscar mediante un punto de inicio un punto fin con menor costo

In [2]:
#Importar librerias
import numpy as np

In [3]:
#Crear la clase Nodo
class Nodo :
    #padre es de tipo nodo y se encarga de referenciar a los nodos ancestros
    #g es el costo
    #h es el costo heuristico estimado para el nodo actual
    #f es el costo total f = g + h

    def __repr__(self):
        return "<Test g:%s h:%s f:%s posicion:%s>" % (self.g, self.h, self.f,self.posicion)

    def __str__(self):
        return "From str <Test g:%s h:%s f:%s posicion:%s>" % (self.g, self.h, self.f,self.posicion)
    
    def __init__(self, padre=None, posicion=None):
        self.padre = padre
        self.posicion = posicion

        self.g = 0
        self.h = 0
        self.f = 0

    def __eq__(self, other):
        return self.posicion == other.posicion

In [4]:
#variable Generica de movimiento basada en el ejercicio
#agrego los movimientos que debe realizar en los casos en los que se encuentrre con los valores
#arriba,izquierda,abajo,derecha
movimientos = [[-1, 0],  # arriba
    [0, -1],  # izquierda
    [1, 0],  # abajo
    [0, 1]]  # derecha

In [5]:
#metodo que se encarga de crear un objeto de tipo nodo usando la posicion
def crear_nodo_por_posicion(posicion):
    # crea nodo de inicio teniendo en cuenta
    nodo_resultante = Nodo(None, tuple(posicion))
    # Asigna el costo,costo heuristico estimado, del nodo inicio
    nodo_resultante.g = 0
    nodo_resultante.h = 0
    nodo_resultante.f = 0

    return nodo_resultante

In [6]:
#utilidad que muestra graficamente la salida al objetivo
def mostrar_ruta(ruta):
    for i in ruta:
        listel = []
        for j in i:
            if j >-1:
                listel.append("@")
            else:
                listel.append("X")
        print(listel)

In [7]:
#Obtiene la nueva posicion de un elemento y de sus hijos
def obtener_posicion( hijos, laberinto, nodo_actual, num_columnas, num_filas):
    for nueva_posicion in movimientos:
        # obtener la posicion del nodo
        posicion_nodo = (
            nodo_actual.posicion[0] + nueva_posicion[0], nodo_actual.posicion[1] + nueva_posicion[1])

        if (posicion_nodo[0] > (num_filas - 1) or
                posicion_nodo[0] < 0 or
                posicion_nodo[1] > (num_columnas - 1) or
                posicion_nodo[1] < 0):
            continue

        # verificar si es viable el nodo en el camino es decir si se puede camianr alli
        if laberinto[posicion_nodo[0]][posicion_nodo[1]] != 0:
            continue

        # crea un nuevo nodo
        nuevo_nodo = Nodo(nodo_actual, posicion_nodo)

        # lo agrega a los nodos hijos
        hijos.append(nuevo_nodo)

In [8]:
def obtener_complejidad_euclidea( costo, hijos, nodo_actual, nodo_fin, puntos_por_visitar, puntos_visitados):
    for hijo in hijos:
        # valida si ya se ha visitado el hijo (en la lista de hijos)
        if len([h_visitado for h_visitado in puntos_visitados if h_visitado == hijo]) > 0:
            continue

        # creamos el valor de los costos para los hijos

        # agregamos el costo
        hijo.g = nodo_actual.g + costo

        # calculamos el costo heuristico usando la  distancia euclidea
        hijo.h = (((hijo.posicion[0] - nodo_fin.posicion[0]) ** 2) +
                  ((hijo.posicion[1] - nodo_fin.posicion[1]) ** 2))

        # se calcula f que es igual a  f = g+h
        hijo.f = hijo.g + hijo.h

        # se verifica si el hijo ya esta en los puntos visitados y si su costo es el mas bajo
        if len([i for i in puntos_por_visitar if hijo == i and hijo.g > i.g]) > 0:
            continue

        # agrega el nodo hijo a la lista
        puntos_por_visitar.append(hijo)

In [9]:
# retorna el camino de la busqueda
def retorna_ruta(nodo_actual, laberinto):
    camino = []
    # se usa el metodo np shape que retorna los valores de la matriz el ancho y el alto
    # en ete caso el numero de filas y columnas
    # se ingresan los valores de la tupla en las variables
    num_filas, num_columnas = np.shape(laberinto)
    # se rellenan las columnas de -1 en las 2 dimenciones
    resultado = [[-1 for i in range(num_columnas)] for j in range(num_filas)]
    # se procesa la coleccion de nodos y se crea una coleccion de posiciones
    actual = nodo_actual

    while actual is not None:
        camino.append(actual.posicion)
        actual = actual.padre
    camino = camino[::-1]
    # se declara un incice para marcar los que cumplen la condicion y hacen parte de la solucion
    # asi va incrementando de 1 en 1
    valor_indice = 0
    for i in range(len(camino)):
        resultado[camino[i][0]][camino[i][1]] = valor_indice
        valor_indice += 1
    return resultado

In [10]:
def busqueda_laberinto(laberinto, costo, inicio, fin):
    # crea nodo de inicio teniendo en cuenta
    nodo_inicio = crear_nodo_por_posicion(inicio)
    # crea nodo de Fin teniendo en cuenta
    nodo_fin = crear_nodo_por_posicion(fin)
    # se crea la lista de puntos visitados
    puntos_visitados = []
    # se crea la lista de puntos por visitar
    puntos_por_visitar = []
    # agregamos el punto de inicio a los puntos por visitar
    puntos_por_visitar.append(nodo_inicio)
    # se crea la variable de iteracion o el contador de iteracion
    iretador_salida = 0
    iteraciones_maximas = (len(laberinto) // 2) ** 10
    # se agrega una validacion de ejecucion paraa no caer en un bucle infinito

    # extraemos las dimenciones del laberinto
    num_filas, num_columnas = np.shape(laberinto)
    # ejecuta elbucle de la busqueda
    while len(puntos_por_visitar) > 0:
        iretador_salida += 1
        # obtener el nodo Actual
        nodo_actual = puntos_por_visitar[0]
        indice_actual = 0
        #print(enumerate(puntos_por_visitar))

        for indice, item in enumerate(puntos_por_visitar):
            #print(puntos_por_visitar)
            #print(item)
            #print(nodo_actual)
            # si el costo total del item es menor que el costo total del item se asigna item
            if item.f < nodo_actual.f:
                nodo_actual = item
                indice_actual = indice

        if (iretador_salida > iteraciones_maximas):
            print("Warning :::: el laberinto presenta muchas iteraciones el costo computacional es muy alto")
            return retorna_ruta(nodo_actual, laberinto)

        # elimina el punto de puntos por visitar
        # lo agrega en puntos visitados
        puntos_por_visitar.pop(indice_actual)
        puntos_visitados.append(nodo_actual)

        if nodo_actual == nodo_fin:
            return retorna_ruta(nodo_actual, laberinto)

        # Se generan nodos hijos apartir de los cuadrados adjacentes
        hijos = []

        obtener_posicion(hijos, laberinto, nodo_actual, num_columnas, num_filas)

        obtener_complejidad_euclidea(costo, hijos, nodo_actual, nodo_fin, puntos_por_visitar, puntos_visitados)

In [11]:
##Test
inicio = [0, 0]  # posicion de inicio 
fin  = [4, 5]  # posicion final
costo = 1  # costo por movimiento
#ruta de laberinto o simplemente laberinto
laberinto = [[0, 1, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0],
            [0, 1, 0, 1, 0, 0],
            [0, 1, 0, 0, 1, 0],
            [0, 0, 0, 0, 1, 0]]

#en este caso 1 es la representacion de uan pared
respuesta = busqueda_laberinto(laberinto, costo, inicio, fin)

#lo graficamos
mostrar_ruta(respuesta)




['@', 'X', 'X', 'X', 'X', 'X']
['@', '@', '@', '@', '@', 'X']
['X', 'X', 'X', 'X', '@', '@']
['X', 'X', 'X', 'X', 'X', '@']
['X', 'X', 'X', 'X', 'X', '@']
